In [ ]:
import sys
import time
from pathlib import Path

import h5py as h5
import numpy as np
from scipy.ndimage import uniform_filter1d

sys.path.append('../lib')
from storage import get_storage_functions
from local_paths import preproc_dir

In [ ]:
#============================================================================
# session
#============================================================================
sess_name = 'sess_name'

#============================================================================
# main
#============================================================================
sdf_method = 'mwa'  # moving window average
sdf_window = 50     # ms

#============================================================================
# paths
#============================================================================
proc_dir   = preproc_dir
output_dir = preproc_dir

# Check parameters and whether already done

In [ ]:
rasters_path = Path(proc_dir) / (sess_name + '-rasters.nwb')
print('Loading rasters from', rasters_path)
rasters_path = rasters_path.expanduser()
assert rasters_path.is_file()

proc_path = Path(proc_dir) / (sess_name + '-proc.h5')
print('Loading shared processing from', proc_path)
proc_path = proc_path.expanduser()
assert proc_path.is_file()

assert sdf_method == 'mwa'  # only using mwa, but can implement others like gaussian filtering

output_dir = Path(output_dir)
assert output_dir.expanduser().is_dir()
output_path = output_dir / (sess_name + f'-sdf-{sdf_method}_{sdf_window}.h5')
print('Saving results to', output_path)
output_path = output_path.expanduser()

In [ ]:
if output_path.is_file():
    with h5.File(output_path, 'r') as f:
        try:
            if f['progress_report/sdf/all_done'][()].item():
                raise RuntimeError(f'{sess_name} has already been processed')
        except KeyError:
            pass

In [ ]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path)

In [ ]:
save_results('config/sdf/method', sdf_method)
save_results('config/sdf/window', sdf_window, attrs=dict(unit='ms'))

# Main

In [ ]:
with h5.File(rasters_path, 'r') as f:
    all_unit_names = f['processing/ecephys/unit_names/unit_name'][()].astype(str)
with h5.File(proc_path, 'r') as f:
    unit_names = f['unit_selection/simple'][()].astype(str)
unexpected_units = set(unit_names) - set(all_unit_names)
assert not len(unexpected_units), f'got {len(unexpected_units)} units in shared processing file absent from rasters data'
unit_sel = np.array([v in unit_names for v in all_unit_names])
unit_sel = np.nonzero(unit_sel)[0]

with h5.File(rasters_path, 'r') as f:
    rasters = f['processing/ecephys/rasters/data'][()]
rasters = rasters[:,unit_sel]

In [ ]:
sdf = rasters.astype(np.float32) * 1e3
if sdf_window == 1:
    pass
else:
    print(f'calculating sdf with a {sdf_window} ms moving average window...', end=' ')
    t0 = time.time()
    sdf = uniform_filter1d(sdf, sdf_window, axis=0)
    print(f'done ({time.time()-t0:.1f} s)')
    print()

print('spike density function:', sdf.shape, sdf.dtype)

In [ ]:
save_results('sdf', sdf, attrs=dict(
    unit_names=unit_names.astype(bytes),
    unit='spikes/s',
    smoothing_method=sdf_method,
    smoothing_window=sdf_window))

# Wrap up

In [ ]:
save_results(f'progress_report/sdf/all_done', True)

In [ ]:
%load_ext watermark
%watermark -vm --iversions -rbg